In [1]:
%load_ext autotime

import os, re, sys, math, pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pdb

from tqdm import tqdm, trange
warnings.filterwarnings('ignore')

In [2]:
import xml.etree.cElementTree as et

path = '../data/raw_test/'
res_path = '../data/derived_test/30_min_no_interpolate/'
parsedXML = et.parse(path+'596-ws-testing.xml')

time: 79 ms


In [3]:
all_params = list(parsedXML.getroot())

time: 359 µs


In [4]:
print (all_params)

[<Element 'glucose_level' at 0x7f1468a0d9f8>, <Element 'finger_stick' at 0x7f146881f5e8>, <Element 'basal' at 0x7f14687a8098>, <Element 'temp_basal' at 0x7f14687a8a98>, <Element 'bolus' at 0x7f14687a8b38>, <Element 'meal' at 0x7f14687b07c8>, <Element 'sleep' at 0x7f14687ba958>, <Element 'work' at 0x7f14687bacc8>, <Element 'stressors' at 0x7f14687bad18>, <Element 'hypo_event' at 0x7f14687bad68>, <Element 'illness' at 0x7f14687baf98>, <Element 'exercise' at 0x7f14687c0048>, <Element 'basis_heart_rate' at 0x7f14687c03b8>, <Element 'basis_gsr' at 0x7f14687c0408>, <Element 'basis_skin_temperature' at 0x7f146823f228>, <Element 'basis_air_temperature' at 0x7f1467ca6bd8>, <Element 'basis_sleep' at 0x7f1467ca6c28>, <Element 'acceleration' at 0x7f1467cad048>]
time: 7.56 ms


In [5]:
temp_dict = {}
for item in all_params:
    temp_dict[item.tag] = []
    for value in item:
        temp_dict[item.tag].append(value.attrib)

time: 12.7 ms


In [6]:
list_dict = []
for k, v in temp_dict.items():
    temp_dict = pd.DataFrame(v)
    if (list(temp_dict.columns) == ['ts_begin', 'ts_end', 'quality']):
        temp_dict['ts'] = temp_dict['ts_begin']
        temp_dict.drop(['ts_begin'], axis = 1, inplace = True)
    elif 'ts_begin' in temp_dict.columns:
        temp_dict['ts'] = temp_dict['ts_begin']
        temp_dict.drop(['ts_begin', 'ts_end'], axis = 1, inplace = True)
    elif 'tbegin' in temp_dict.columns:
        temp_dict['ts'] = temp_dict['tbegin']
        temp_dict.drop(['tbegin', 'tend'], axis = 1, inplace = True)
    list_dict.append(pd.DataFrame(temp_dict))

time: 35 ms


In [7]:
len_list = len(list_dict)
i = 0
while i<len_list:
    if len(list_dict[i].columns)<=1:
        list_dict.pop(i)
    else:
        i += 1
    len_list = len(list_dict)
    
# list_dict.pop(8)
# list_dict.pop(8)

time: 741 µs


In [8]:
updated_list_dict = []
for dictionary in list_dict:
    dictionary['ts'] = pd.to_datetime(dictionary['ts'], format = '%d-%m-%Y %H:%M:%S')
    dictionary.index = dictionary['ts']
    dictionary.drop(['ts'], axis = 1, inplace = True)
    
    for i in dictionary.columns:
        try:
            dictionary[i] = dictionary[i].astype('float')
        except:
            pass
        
    cat_cols = ['type']
    _df = pd.DataFrame(dictionary)
    try:
        _df = pd.get_dummies(columns = cat_cols, data = _df)
        df = _df.resample('30T').mode()
    except:
        df = _df.resample('30T').mean()
    
    print (df)
    df = df.reindex(pd.date_range('2027-05-27 00:00:00', '2027-06-06 10:00:00', freq='30T'))
    '''
    if ('carbs' in df.columns) or (list(df.columns) == ['quality']):
        pass
    else:
        df = df.interpolate()
    '''
    print (df.columns)
    updated_list_dict.append(df)

                          value
ts                             
2027-05-27 00:00:00  171.833333
2027-05-27 00:30:00  166.833333
2027-05-27 01:00:00  163.833333
2027-05-27 01:30:00  163.500000
2027-05-27 02:00:00  167.833333
...                         ...
2027-06-06 08:00:00  132.333333
2027-06-06 08:30:00  121.833333
2027-06-06 09:00:00   97.833333
2027-06-06 09:30:00   88.166667
2027-06-06 10:00:00   87.666667

[501 rows x 1 columns]
Index(['value'], dtype='object')
                     value
ts                        
2027-05-27 05:30:00  183.0
2027-05-27 06:00:00    NaN
2027-05-27 06:30:00    NaN
2027-05-27 07:00:00    NaN
2027-05-27 07:30:00    NaN
...                    ...
2027-06-05 15:30:00    NaN
2027-06-05 16:00:00    NaN
2027-06-05 16:30:00    NaN
2027-06-05 17:00:00    NaN
2027-06-05 17:30:00   75.0

[457 rows x 1 columns]
Index(['value'], dtype='object')
                     value
ts                        
2027-05-27 00:00:00    0.4
2027-05-27 00:30:00    NaN
2027-05-27 

In [9]:
updated_list_dict[0].columns = ['gl_value']
updated_list_dict[1].columns = ['fs_value']
updated_list_dict[2].columns = ['basel_value']
updated_list_dict[3].columns = ['tb_value']
updated_list_dict[4].columns = ['bolus_dose', 'bolus_type_normal']
updated_list_dict[5].columns = ['meal_carbs', 'meal_type_Breakfast', 'meal_type_Dinner', 'meal_type_HypoCoreection', 'meal_type_Lunch', 'meal_type_Snack']
updated_list_dict[6].columns = ['sleep_quality']
# updated_list_dict[6].columns = ['work_intensity']
updated_list_dict[7].columns = ['exercise_intensity', 'exercise_duration', 'exercise_type']
# updated_list_dict[9].columns = ['basis_heart_rate']
updated_list_dict[8].columns = ['basis_gsr']
updated_list_dict[9].columns = ['basis_skin_temp']
# updated_list_dict[12].columns = ['basis_air_temp']
# updated_list_dict[13].columns = ['basis_steps']
updated_list_dict[10].columns = ['basis_sleep_quality', 'basis_sleep_type']
updated_list_dict[11].columns = ['acceleration_value']

time: 2.48 ms


In [10]:
dataframe = updated_list_dict[0].join(updated_list_dict[1], how = 'outer')

for i in trange(1,len(updated_list_dict)-1):
    dataframe = dataframe.join(updated_list_dict[i+1], how = 'outer')

100%|██████████| 10/10 [00:00<00:00, 1623.18it/s]

time: 15.3 ms


In [11]:
dataframe.isnull().sum()

gl_value                     38
fs_value                    469
basel_value                 470
tb_value                    500
bolus_dose                  463
bolus_type_normal           463
meal_carbs                  447
meal_type_Breakfast         447
meal_type_Dinner            447
meal_type_HypoCoreection    447
meal_type_Lunch             447
meal_type_Snack             447
sleep_quality               491
exercise_intensity          491
exercise_duration           491
exercise_type               491
basis_gsr                   166
basis_skin_temp             166
basis_sleep_quality         491
basis_sleep_type            491
acceleration_value          166
dtype: int64

time: 3.95 ms


In [12]:
dataframe

,gl_value,fs_value,basel_value,tb_value,bolus_dose,bolus_type_normal,meal_carbs,meal_type_Breakfast,meal_type_Dinner,meal_type_HypoCoreection,...,meal_type_Snack,sleep_quality,exercise_intensity,exercise_duration,exercise_type,basis_gsr,basis_skin_temp,basis_sleep_quality,basis_sleep_type,acceleration_value
2027-05-27 00:00:00,171.833333,NaN,0.4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2027-05-27 00:30:00,166.833333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2027-05-27 01:00:00,163.833333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2027-05-27 01:30:00,163.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2027-05-27 02:00:00,167.833333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2027-06-06 08:00:00,132.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.019382,76.028000,NaN,NaN,1.069814
2027-06-06 08:30:00,121.833333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.005069,55.947500,NaN,NaN,0.829836
2027-06-06 09:00:00,97.833333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2027-06-06 09:30:00,88.166667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.029690,80.076538,NaN,NaN,1.051070


time: 28.2 ms


In [13]:
dataframe.to_csv(res_path + '596-ws-testing.csv')

time: 8.54 ms
